<a href="https://www.kaggle.com/code/abd0kamel/mutemotion-asl-citizen-translation-model?scriptVersionId=158754249" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Introduction**

<h3> This notebook aims to use MediaPipe landmarks detection as the starting point
    
<h3> for building an American Sign Language Translation Model using <b>ASL Citizen</b> Dataset
            
**<h3> Sections:**
* [Importing Libraries](#Importing_Libraries)
* [Data Preparation](#Data_Preparation)
* [MediaPipe Implementation](#MediaPipe_Implementation)
* [Visualizing Landmarks](#Visualizing_Landmarks)
* [Data Encoding](#Data_Encoding)
* [Label Filtering](#Label_Filtering)
* [Data Loading](#Data_Loading) --> (Training Start Here)
* [Data Augmentation](#Data_Augmentation)
* [Data Preprocessing](#Data_Preprocessing)
* [Label Encoding](#Label_Encoding)
* [Model Training](#Model_Training)
* [Model Visualization](#Model_Visualization)

---

<div id="Importing_Libraries">
    
# **Importing Libraries** 

In [1]:
!pip install -q mediapipe==0.10.7

In [2]:
import os
import gc
import cv2
import json
import time
import shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import mediapipe as mp
import tensorflow as tf
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from IPython.display import clear_output, FileLink

---

<div id="Data_Preparation">
    
# **Data Preparation**
**<h2>(Done Once)**

In [ ]:
train_path = '/kaggle/input/asl-citizen/ASL_Citizen/splits/train.csv'
val_path = '/kaggle/input/asl-citizen/ASL_Citizen/splits/val.csv'
test_path = '/kaggle/input/asl-citizen/ASL_Citizen/splits/test.csv'

train_data = pd.read_csv(train_path)
val_data = pd.read_csv(val_path)
test_data = pd.read_csv(test_path)

In [ ]:
train_data['split'] = 'train'
val_data['split'] = 'val'
test_data['split'] = 'test'

train_data = train_data[['Video file', 'Gloss', 'split']]
val_data = val_data[['Video file', 'Gloss', 'split']]
test_data = test_data[['Video file', 'Gloss', 'split']]

data = pd.concat([train_data, val_data, test_data], ignore_index=True)
data = data.sample(frac=1, random_state=42, ignore_index=True)
data = data.rename(columns={'Gloss':'label', 'Video file': 'video'})
data.head()

In [ ]:
def extract_label(file_name):
    label_parts = file_name.split("-")[1].split(".")[0].replace("_", " ").split(" ")
    label_parts = [part for part in label_parts if part != '']
    if label_parts[0].startswith('seed'):
        label_parts[0] = label_parts[0][4:]
    if label_parts[-1].isdigit():
        label_parts = label_parts[:-1]
    label_parts = [part.lower() for part in label_parts]
    return ' '.join(label_parts)

In [ ]:
data.insert(data.columns.get_loc('label') + 1, 'clean label', data['video'].apply(extract_label))
data.head()

In [ ]:
data[data['clean label'] == '']

In [ ]:
data.iloc[59741, data.columns.get_loc('clean label')] = 'sour'
data[data['clean label'] == '']

In [ ]:
data.to_csv("data.csv", index=False)

---

<div id="MediaPipe_Implementation">

# **MediaPipe Implementation**
**<h2>(Not important when training)**

In [ ]:
filtered_hand = list(range(21))

filtered_pose = [11, 12, 13, 14, 15, 16]

filtered_face = [0, 4, 7, 8, 10, 13, 14, 17, 21, 33, 37, 39, 40, 46, 52, 53, 54, 55, 58,
                 61, 63, 65, 66, 67, 70, 78, 80, 81, 82, 84, 87, 88, 91, 93, 95, 103, 105,
                 107, 109, 127, 132, 133, 136, 144, 145, 146, 148, 149, 150, 152, 153, 154,
                 155, 157, 158, 159, 160, 161, 162, 163, 172, 173, 176, 178, 181, 185, 191,
                 234, 246, 249, 251, 263, 267, 269, 270, 276, 282, 283, 284, 285, 288, 291,
                 293, 295, 296, 297, 300, 308, 310, 311, 312, 314, 317, 318, 321, 323, 324,
                 332, 334, 336, 338, 356, 361, 362, 365, 373, 374, 375, 377, 378, 379, 380,
                 381, 382, 384, 385, 386, 387, 388, 389, 390, 397, 398, 400, 402, 405, 409,
                 415, 454, 466, 468, 473]

HAND_NUM = len(filtered_hand)
POSE_NUM = len(filtered_pose)
FACE_NUM = len(filtered_face)

In [ ]:
hands = mp.solutions.hands.Hands()
pose = mp.solutions.pose.Pose()
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

def get_frame_landmarks(frame):
    
    all_landmarks = np.zeros((HAND_NUM * 2 + POSE_NUM + FACE_NUM, 3))
    
    def get_hands(frame):
        results_hands = hands.process(frame)
        if results_hands.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results_hands.multi_hand_landmarks):
                if results_hands.multi_handedness[i].classification[0].index == 0: 
                    all_landmarks[:HAND_NUM, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]) # right
                else:
                    all_landmarks[HAND_NUM:HAND_NUM * 2, :] = np.array(
                        [(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark]) # left

    def get_pose(frame):
        results_pose = pose.process(frame)
        if results_pose.pose_landmarks:
            all_landmarks[HAND_NUM * 2:HAND_NUM * 2 + POSE_NUM, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_pose.pose_landmarks.landmark])[filtered_pose]
        
    def get_face(frame):
        results_face = face_mesh.process(frame)
        if results_face.multi_face_landmarks:
            all_landmarks[HAND_NUM * 2 + POSE_NUM:, :] = np.array(
                [(lm.x, lm.y, lm.z) for lm in results_face.multi_face_landmarks[0].landmark])[filtered_face]
        
    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.submit(get_hands, frame)
        executor.submit(get_pose, frame)
        executor.submit(get_face, frame)

    return all_landmarks

In [ ]:
def get_video_landmarks(video_path):
    cap = cv2.VideoCapture(video_path)
    all_frame_landmarks = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_landmarks = get_frame_landmarks(frame)
        all_frame_landmarks.append(frame_landmarks)

    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.submit(cap.release)
        executor.submit(hands.reset)
        executor.submit(pose.reset)
        executor.submit(face_mesh.reset)
        
    return np.array(all_frame_landmarks)

In [ ]:
def draw_landmarks(input_path, output_path, video_landmarks):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_landmarks = video_landmarks[frame_index]
        landmarks = [(int(x * width), int(y * height)) for x, y, _ in frame_landmarks]
        for x, y in landmarks:
            cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
        out.write(frame)
        frame_index += 1

    cap.release()
    out.release()

---

<div id="Visualizing_Landmarks">

# **Visualizing Landmarks**
**<h2>(Not important when training)**

In [ ]:
import requests
from PIL import Image
from io import BytesIO

image_url = 'https://images.unsplash.com/photo-1515294898968-a408405d7674'
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))
img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

plt.figure(figsize=(8, 8))
plt.imshow(img[:,:,::-1])
plt.axis('off')
plt.show()

In [ ]:
height, width, _ = img.shape

frame_landmarks = get_frame_landmarks(img[:,:,::-1])
for landmark in frame_landmarks:
    x = int(landmark[0] * width)
    y = int(landmark[1] * height)
    cv2.circle(img, (x, y), 10, (0, 255, 0), -1)

plt.figure(figsize=(8, 8))
plt.imshow(img[:,:,::-1])
plt.axis('off')
plt.show()

In [ ]:
test = data.iloc[75]
video_dir = '/kaggle/input/asl-citizen/ASL_Citizen/videos'
input_path = os.path.join(video_dir, test['video'])
video_landmarks = get_video_landmarks(input_path)

output_path = '/kaggle/working/landmarks_test.mp4'
draw_landmarks(input_path, output_path, video_landmarks)

---

<div id="Data_Encoding">
    
# **Data Encoding**
**<h2>(Done Once)**

In [ ]:
txt_path = '/kaggle/working/completed.txt'
video_dir = '/kaggle/input/asl-citizen/ASL_Citizen/videos'
npy_dir = '/kaggle/working/landmarks'
os.makedirs(npy_dir, exist_ok=True)

In [ ]:
labels = []
with open('/kaggle/working/Filtered Labels/filtered_labels_221.txt', 'r') as file:
    labels = file.read().splitlines()
    
len(labels)

In [ ]:
data = pd.read_csv('/kaggle/input/asl-citizen-encoded/Data/data.csv')
data = data[data['clean label'].isin(labels)].reset_index(drop=True)
data.head()

In [ ]:
len(data)

In [ ]:
try:
    completed_files = set()
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as file:
            completed_files = {line.strip() for line in file}
            
    for i in tqdm(range(len(data)), ncols=100):
        video_name = data.iloc[i]['video'].split('.')[0]
        if video_name in completed_files: continue
        npy_path = os.path.join(npy_dir, f'{video_name}.npy')
        video_path = os.path.join(video_dir, f'{video_name}.mp4')

        try:
            video_landmarks = get_video_landmarks(video_path)
            np.save(npy_path, video_landmarks)
            
            with open(txt_path, 'a') as file:
                file.write(f"{video_name}\n")
            completed_files.add(video_name)
            
        except Exception as e:
            print(f"\nError encoding {i}: {video_path}\n{e}")
            continue   
        clear_output(wait=True)

except KeyboardInterrupt:
    print("\nLoading process interrupted by user.")

In [ ]:
landmarks_dict = {}

for filename in os.listdir(npy_dir):
    if filename.endswith('.npy'):
        key = os.path.splitext(filename)[0]
        landmarks = np.load(os.path.join(npy_dir, filename), allow_pickle=True)
        landmarks_dict[key] = landmarks

np.savez_compressed('/kaggle/working/landmarks.npz', **landmarks_dict)

FileLink(r'landmarks.npz')

In [ ]:
# shutil.rmtree(npy_dir)
# os.remove('/kaggle/working/completed.txt')
# os.remove('/kaggle/working/landmarks_test.mp4')

---

<div id="Label_Filtering">

# **Label Filtering**
**<h2>(Done Once)**

In [ ]:
# import fasttext
# import fasttext.util

# fasttext.util.download_model('en', if_exists='ignore')

In [ ]:
import fasttext

ft = fasttext.load_model('cc.en.300.bin')

In [ ]:
data = pd.read_csv('/kaggle/input/asl-citizen-encoded/Data/data.csv')
labels = data['clean label'].unique().tolist()
len(data), len(labels)

In [ ]:
labels_dir = '/kaggle/working/Vector Encoded Labels'
os.makedirs(labels_dir, exist_ok=True)

for i in tqdm(range(len(labels)), ncols=100):
    label_vector = ft.get_sentence_vector(labels[i])
    label_path = os.path.join(labels_dir, f'{labels[i]}.npy')
    np.save(label_path, label_vector)

In [ ]:
labels_dir = '/kaggle/input/asl-citizen-encoded/Vector Encoded Labels'
encoded_labels = {}

for filename in os.listdir(labels_dir):
    if filename.endswith('.npy'):
        label = os.path.splitext(filename)[0]
        encoded_label = np.load(os.path.join(labels_dir, filename), allow_pickle=True)
        encoded_labels[label] = encoded_label
        
len(encoded_labels)

In [ ]:
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
word = 'taxi'
word_vector = ft.get_sentence_vector(word)

similarity_scores = []
for label, label_vector in encoded_labels.items():
    similarity_scores.append(cosine_similarity(word_vector, label_vector))

In [ ]:
bins = np.linspace(0, 1, 21)

plt.hist(similarity_scores, bins=bins, edgecolor='black')
plt.title(f'Cosine Similarity Distribution to "{word}"')
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')
plt.xticks(np.arange(0, 1.1, 0.1))
plt.grid(axis='y')
plt.show()

In [ ]:
threshold = 0.4
similar_words_set = set()

for label, label_vector in encoded_labels.items():
    similarity_score = cosine_similarity(word_vector, label_vector)
    if similarity_score > threshold:
        similar_words_set.add(label)
        
print(similar_words_set)

---

In [ ]:
taxi_words = [
    'hello',
    'goodbye',
    'help',
    'thank you',
    'sorry',
    'yes',
    'no',
    'stop',
    'go',
    'left',
    'right',
    'ahead',
    'around',
    'address',
    'destination',
    'time',
    'money',
    'cost',
    'lost',
    'map',
    'street',
    'road',
    'taxi',
    'car',
    'bus',
    'school',
    'university',
    'food',
    'fuel',
]

len(taxi_words)

In [ ]:
taxi_dir = '/kaggle/working/Vector Encoded Samples'
os.makedirs(taxi_dir, exist_ok=True)

for i in tqdm(range(len(taxi_words)), ncols=100):
    taxi_vector = ft.get_sentence_vector(taxi_words[i])
    taxi_path = os.path.join(taxi_dir, f'{taxi_words[i]}.npy')
    np.save(taxi_path, taxi_vector)

In [ ]:
taxi_dir = '/kaggle/input/asl-citizen-encoded/Vector Encoded Samples'
taxi_vectors = []

for filename in os.listdir(taxi_dir):
    if filename.endswith('.npy'):
        taxi_vectors.append(np.load(os.path.join(taxi_dir, filename), allow_pickle=True))
        
len(taxi_vectors)

In [ ]:
threshold = 0.4
similar_words_set = set()

for label, label_vector in encoded_labels.items():
    for taxi_vector in taxi_vectors:
        similarity_score = cosine_similarity(taxi_vector, label_vector)
        if similarity_score > threshold:
            similar_words_set.add(label)
            
similar_words_list = list(similar_words_set)

In [ ]:
len(similar_words_list)

In [ ]:
with open(f'/kaggle/working/Filtered Labels/filtered_labels_{len(similar_words_list)}.txt', 'w') as file:
    for label in similar_words_list:
        file.write(label + '\n')

In [ ]:
del ft
gc.collect()

---

<div id="Data_Loading">
    
# **Data Loading**

In [3]:
data = pd.read_csv('/kaggle/input/asl-citizen-encoded/Data/data.csv')
len(data)

83399

In [4]:
labels = []
with open('/kaggle/working/Filtered Labels/filtered_labels_221.txt', 'r') as file:
    labels = file.read().splitlines()
len(labels)

221

In [5]:
filtered_hand = list(range(21))

filtered_pose = [11, 12, 13, 14, 15, 16]

filtered_face = [0, 4, 7, 8, 10, 13, 14, 17, 21, 33, 37, 39, 40, 46, 52, 53, 54, 55, 58,
                 61, 63, 65, 66, 67, 70, 78, 80, 81, 82, 84, 87, 88, 91, 93, 95, 103, 105,
                 107, 109, 127, 132, 133, 136, 144, 145, 146, 148, 149, 150, 152, 153, 154,
                 155, 157, 158, 159, 160, 161, 162, 163, 172, 173, 176, 178, 181, 185, 191,
                 234, 246, 249, 251, 263, 267, 269, 270, 276, 282, 283, 284, 285, 288, 291,
                 293, 295, 296, 297, 300, 308, 310, 311, 312, 314, 317, 318, 321, 323, 324,
                 332, 334, 336, 338, 356, 361, 362, 365, 373, 374, 375, 377, 378, 379, 380,
                 381, 382, 384, 385, 386, 387, 388, 389, 390, 397, 398, 400, 402, 405, 409,
                 415, 454, 466, 468, 473]

HAND_NUM = len(filtered_hand)
POSE_NUM = len(filtered_pose)
FACE_NUM = len(filtered_face)

In [6]:
landmarks = (
    [x for x in filtered_hand] +
    [x + HAND_NUM for x in filtered_hand] +
    [x + HAND_NUM * 2 for x in filtered_pose] +
    [x + HAND_NUM * 2 + POSE_NUM for x in filtered_face]
)

print(landmarks)
print(f'\nTotal Number: {len(landmarks)}')

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 53, 54, 55, 56, 57, 58, 48, 52, 55, 56, 58, 61, 62, 65, 69, 81, 85, 87, 88, 94, 100, 101, 102, 103, 106, 109, 111, 113, 114, 115, 118, 126, 128, 129, 130, 132, 135, 136, 139, 141, 143, 151, 153, 155, 157, 175, 180, 181, 184, 192, 193, 194, 196, 197, 198, 200, 201, 202, 203, 205, 206, 207, 208, 209, 210, 211, 220, 221, 224, 226, 229, 233, 239, 282, 294, 297, 299, 311, 315, 317, 318, 324, 330, 331, 332, 333, 336, 339, 341, 343, 344, 345, 348, 356, 358, 359, 360, 362, 365, 366, 369, 371, 372, 380, 382, 384, 386, 404, 409, 410, 413, 421, 422, 423, 425, 426, 427, 428, 429, 430, 432, 433, 434, 435, 436, 437, 438, 445, 446, 448, 450, 453, 457, 463, 502, 514, 516, 521]

Total Number: 180


In [7]:
def load_data(split, *paths, labels=None, landmarks=None):
    if labels is None:
        labels = set(data['clean label'].unique().tolist())

    filtered_data = data[(data['split'] == split.lower()) & (data['clean label'].isin(labels))]
    label_dict = dict(zip(filtered_data['video'].str.replace('.mp4', ''), filtered_data['clean label']))
    video_set = set(label_dict.keys())

    X, Y = [], []
    for path in paths:
        landmarks_dict = np.load(path, allow_pickle=True)
        keys = [k for k in landmarks_dict.keys() if k in video_set]

        if landmarks is None:
            landmarks = list(range(landmarks_dict[keys[0]].shape[1]))

        split_name = {'train': 'Training', 'test': 'Validation', 'val': 'Test'}[split.lower()]
        print(f"\nLoading {split_name} data from {path}:")
        
        for k in tqdm(keys, ncols=100):
            X.append(landmarks_dict[k][:, landmarks, :])
            Y.append(label_dict.get(k, None))

    return X, Y

In [8]:
X_train, Y_train = load_data('train', '/kaggle/input/asl-citizen-encoded/landmarks_p1.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p2.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p3.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p4.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p5.npz',
                             labels=labels, landmarks=landmarks)

X_val, Y_val = load_data('test', '/kaggle/input/asl-citizen-encoded/landmarks_p1.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p2.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p3.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p4.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p5.npz',
                             labels=labels, landmarks=landmarks)

X_test, Y_test = load_data('val', '/kaggle/input/asl-citizen-encoded/landmarks_p1.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p2.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p3.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p4.npz',
                             '/kaggle/input/asl-citizen-encoded/landmarks_p5.npz',
                             labels=labels, landmarks=landmarks)


Loading Training data from /kaggle/input/asl-citizen-encoded/landmarks_p1.npz:


100%|█████████████████████████████████████████████████████████████| 740/740 [00:21<00:00, 33.97it/s]



Loading Training data from /kaggle/input/asl-citizen-encoded/landmarks_p2.npz:


100%|█████████████████████████████████████████████████████████████| 693/693 [00:20<00:00, 33.91it/s]



Loading Training data from /kaggle/input/asl-citizen-encoded/landmarks_p3.npz:


...

100%|█████████████████████████████████████████████████████████████| 192/192 [00:05<00:00, 35.01it/s]



Loading Test data from /kaggle/input/asl-citizen-encoded/landmarks_p4.npz:


100%|█████████████████████████████████████████████████████████████| 183/183 [00:05<00:00, 36.52it/s]



Loading Test data from /kaggle/input/asl-citizen-encoded/landmarks_p5.npz:


100%|█████████████████████████████████████████████████████████████| 195/195 [00:05<00:00, 32.82it/s]


In [9]:
len(X_train), len(X_val), len(X_test)

(3531, 2900, 907)

In [10]:
X_train[0].shape

(125, 180, 3)

In [11]:
len(np.unique(Y_train)), len(np.unique(Y_val)), len(np.unique(Y_test))

(221, 221, 221)

In [12]:
np.all(np.in1d(np.unique(Y_val), np.unique(Y_train))),\
np.all(np.in1d(np.unique(Y_test), np.unique(Y_train)))

(True, True)

In [13]:
gc.collect()

595

---

<div id="Data_Augmentation">
    
# **Data Augmentation**

In [ ]:
def rotate(data, rotation_matrix):
    frames, landmarks, _ = data.shape
    center = np.array([0.5, 0.5, 0])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data = data.reshape(-1, 3)
    data[non_zero] -= center
    data[non_zero] = np.dot(data[non_zero], rotation_matrix.T)
    data[non_zero] += center
    data = data.reshape(frames, landmarks, 3)
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def rotate_z(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), -np.sin(theta), 0],
        [np.sin(theta), np.cos(theta), 0],
        [0, 0, 1]
    ])
    return rotate(data, rotation_matrix)

def rotate_y(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

def rotate_x(data):
    angle = np.random.choice([np.random.uniform(-30, -10),
                              np.random.uniform(10, 30)])
    theta = np.radians(angle)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, np.cos(theta), -np.sin(theta)],
        [0, np.sin(theta), np.cos(theta)]
    ])
    return rotate(data, rotation_matrix)

In [ ]:
def zoom(data):
    factor = np.random.uniform(0.8, 1.2)
    center = np.array([0.5, 0.5])
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], :2] = (
        (data[non_zero[:, 0], non_zero[:, 1], :2] - center) * factor + center
    )
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def shift(data):
    x_shift = np.random.uniform(-0.2, 0.2)
    y_shift = np.random.uniform(-0.2, 0.2)
    non_zero = np.argwhere(np.any(data[:, :, :2] != 0, axis=2))
    data[non_zero[:, 0], non_zero[:, 1], 0] += x_shift
    data[non_zero[:, 0], non_zero[:, 1], 1] += y_shift
    out_of_range = np.any((data[:, :, :2] < 0) | (data[:, :, :2] > 1), axis=2)
    data[out_of_range] = 0
    return data

def mask(data):
    frames, landmarks, _ = data.shape
    num_hands = int(0.3 * 42)
    num_rest = int(0.6 * (landmarks - 42))

    mask = np.zeros(landmarks, dtype=bool)
    indices = np.concatenate([
        np.random.choice(42, num_hands, replace=False),
        np.random.choice(landmarks - 42, num_rest, replace=False) + 42
    ])
    mask[indices] = True
    data[:, mask] = 0
    return data

# def hflip(data):
#     data[:, :, 0] = 1 - data[:, :, 0]
#     return data

def speedup(data):
    return data[::2]

In [ ]:
def apply_augmentations(data):
    aug_functions = [rotate_x, rotate_y, rotate_z, zoom, shift, mask, speedup]
    np.random.shuffle(aug_functions)
    counter = 0
    for fun in aug_functions:
        if np.random.rand() < 0.5:
            data = fun(data)
            counter += 1
    
    if counter == 0:
        data = apply_augmentations(data)

    return data

In [ ]:
def augment(X, Y, num=None):
    X_aug = X.copy()
    Y_aug = Y.copy()
    
    if num == None:
        for i in tqdm(range(len(Y)), ncols=100):
            num_aug = np.random.choice([1, 2, 3])
            for n in range(num_aug):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])
    elif num > 0:
        for i in tqdm(range(len(Y)), ncols=100):
            for n in range(num):
                X_aug.append(apply_augmentations(X[i].copy()))
                Y_aug.append(Y[i])

    return X_aug, Y_aug

In [ ]:
X_train, Y_train = augment(X_train, Y_train, num=1)

In [ ]:
len(X_train), len(X_train[0]), len(X_train[0][0]), len(X_train[0][0][0])

In [ ]:
permutation_train = list(range(len(Y_train)))
np.random.shuffle(permutation_train)
X_train = [X_train[i] for i in permutation_train]
Y_train = [Y_train[i] for i in permutation_train]

permutation_val = list(range(len(Y_val)))
np.random.shuffle(permutation_val)
X_val = [X_val[i] for i in permutation_val]
Y_val = [Y_val[i] for i in permutation_val]

permutation_test = list(range(len(Y_test)))
np.random.shuffle(permutation_test)
X_test = [X_test[i] for i in permutation_test]
Y_test = [Y_test[i] for i in permutation_test]

---

<div id="Data_Preprocessing">
    
# **Data Preprocessing**

In [22]:
max(len(x) for x in X_train), max(len(x) for x in X_val), max(len(x) for x in X_test)

(664, 289, 208)

In [23]:
sum(len(x) for x in X_train) / len(X_train),\
sum(len(x) for x in X_val) / len(X_val),\
sum(len(x) for x in X_test) / len(X_test)

(83.72075899178702, 74.6651724137931, 83.87431091510474)

In [24]:
def padding(X, Y, length=None, pad=0):
    if length is None:
        length = max(len(x) for x in X)
    
    X_padded = []
    for x in X:
        if len(x) > length:
            X_padded.append(x[:length]) #truncate
        else:
            pad_length = length - len(x)
            X_padded.append(np.pad(
                x, ((0, pad_length), (0, 0), (0, 0)),
                mode='constant', constant_values=pad
            ))
            
    X_padded = np.array(X_padded)
    Y = np.array(Y)
    return X_padded, Y

In [25]:
X_train, Y_train = padding(X_train, Y_train, length=200, pad=-100)
X_val, Y_val = padding(X_val, Y_val, length=200, pad=-100)
X_test, Y_test = padding(X_test, Y_test, length=200, pad=-100)

In [26]:
X_train.shape, X_val.shape, X_test.shape

((3531, 200, 180, 3), (2900, 200, 180, 3), (907, 200, 180, 3))

---

---

<div id="Label_Encoding">
    
# **Label Encoding**

In [ ]:
# label_encoder = LabelEncoder()

# Y_train = label_encoder.fit_transform(Y_train)
# Y_val = label_encoder.fit_transform(Y_val)
# Y_test = label_encoder.fit_transform(Y_test)

In [ ]:
# labels = np.unique(Y_train)
label_mapping = {label: idx for idx, label in enumerate(labels)}
with open(f'/kaggle/working/Filtered Labels/label_mapping_{len(labels)}.json', 'w') as json_file:
    json.dump(label_mapping, json_file, indent=4)
    
index_mapping = {idx: label for label, idx in label_mapping.items()}
with open(f'/kaggle/working/Filtered Labels/index_mapping_{len(labels)}.json', 'w') as json_file:
    json.dump(index_mapping, json_file, indent=4)

In [ ]:
Y_train= np.array([label_mapping[label] for label in Y_train])
Y_val = np.array([label_mapping[label] for label in Y_val])
Y_test = np.array([label_mapping[label] for label in Y_test])

In [ ]:
Y_train

---

In [ ]:
labels_dir = '/kaggle/input/asl-citizen-encoded/Vector Encoded Labels'
encoded_labels = {}

for filename in os.listdir(labels_dir):
    if filename.endswith('.npy'):
        label = os.path.splitext(filename)[0]
        encoded_label = np.load(os.path.join(labels_dir, filename), allow_pickle=True)
        encoded_labels[label] = encoded_label
        
len(encoded_labels)

In [ ]:
Y_train = np.array([encoded_labels[label] for label in Y_train])
Y_val = np.array([encoded_labels[label] for label in Y_val])
Y_test = np.array([encoded_labels[label] for label in Y_test])

In [ ]:
Y_train[0].shape

In [ ]:
gc.collect()

---

<div id="Model_Training">
    
# **Model Training**

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)
physical_devices

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Masking(mask_value=-100, input_shape=X_train[0].shape),
    tf.keras.layers.Reshape((X_train[0].shape[0], -1)),

    tf.keras.layers.Conv1D(64, 1, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.DepthwiseConv1D(3, strides=1, padding='valid', depth_multiplier=1, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv1D(64, 1, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.DepthwiseConv1D(5, strides=2, padding='valid', depth_multiplier=4, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.MaxPool1D(2, 2),

    tf.keras.layers.Conv1D(256, 1, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.DepthwiseConv1D(3, strides=1, padding='valid', depth_multiplier=1, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Conv1D(256, 1, strides=1, padding='valid', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.DepthwiseConv1D(5, strides=2, padding='valid', depth_multiplier=4, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.GlobalAvgPool1D(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Dense(300)
])

In [ ]:
lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.9
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=100,
    restore_best_weights=True
)

model.compile(
    loss=tf.keras.losses.cosine_similarity,
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
)

In [ ]:
hist = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=1000,
    batch_size=128,
    callbacks=[early_stopping]
)

In [ ]:
# del X_train, Y_train
gc.collect()

In [ ]:
model.evaluate(X_val, Y_val)

In [ ]:
model.evaluate(X_test, Y_test)

---

<div id="Model_Visualization">
    
# **Model Visualization**

In [ ]:
interval = 2
train_loss = hist.history['loss'][::interval]
val_loss = hist.history['val_loss'][::interval]
# train_acc = hist.history['accuracy'][::interval]
# val_acc = hist.history['val_accuracy'][::interval]

# Plotting loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_loss, 'g', label='Training loss')
plt.plot(val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel(f'Epochs x {interval}')
plt.ylabel('Loss')
plt.legend()

# Plotting accuracy
# plt.subplot(1, 2, 2)
# plt.plot(train_acc, 'b', label='Training accuracy')
# plt.plot(val_acc, 'g', label='Validation accuracy')
# plt.title('Training and Validation Accuracy')
# plt.xlabel(f'Epochs x {interval}')
# plt.ylabel('Accuracy')
# plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
# img = plt.imread('model_plot.png')
# plt.figure(figsize=(40, 30))
# plt.imshow(img)
# plt.axis('off')
# plt.show()

In [ ]:
name = '1-12'
model_filepath = '/kaggle/working/Kamel_Models'
os.makedirs(model_filepath, exist_ok=True)

model.save(model_filepath + f'/{name}.h5')

# model.save(model_filepath + f'/{name}')
# converter = tf.lite.TFLiteConverter.from_saved_model(model_filepath + f'/{name}')
# converter.target_spec.supported_ops = [
#   tf.lite.OpsSet.TFLITE_BUILTINS,
#   tf.lite.OpsSet.SELECT_TF_OPS
# ]
# tflite_model = converter.convert()
# with open(model_filepath + f'/{name}.tflite', 'wb') as f:
#     f.write(tflite_model)

# shutil.rmtree(name)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

Y_pred_probabilities = model.predict(X_test)
Y_pred = np.argmax(Y_pred_probabilities, axis=1)
conf_matrix = confusion_matrix(Y_test, Y_pred)
conf_matrix = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(24, 20))
sns.heatmap(conf_matrix, cmap='Blues', annot=False, square=True)

plt.title('Confusion Matrix for Test Data')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

---

In [103]:
idx = 90
X_test[idx].shape, Y_test[idx]

((200, 180, 3), 'eat')

In [104]:
test_label = Y_test[idx]
test = np.expand_dims(X_test[idx], axis=0)
test.shape

(1, 200, 180, 3)

In [105]:
labels_dir = '/kaggle/input/asl-citizen-encoded/Vector Encoded Labels'
encoded_labels = {}

for filename in os.listdir(labels_dir):
    if filename.endswith('.npy'):
        label = os.path.splitext(filename)[0]
        encoded_label = np.load(os.path.join(labels_dir, filename), allow_pickle=True)
        encoded_labels[label] = encoded_label
        
len(encoded_labels)

2299

In [106]:
labels = np.unique(Y_train)
encoded_labels = {key : encoded_labels[key] for key in encoded_labels.keys() if key in labels}
len(encoded_labels)

221

In [107]:
test_vector = np.array(encoded_labels[test_label])
test_vector.shape

(300,)

In [108]:
model = tf.keras.models.load_model('/kaggle/working/Kamel_Models/1-12-vector.h5')
model.input_shape, model.output_shape

((None, 200, 180, 3), (None, 300))

In [109]:
prediction = model.predict(test)
prediction.shape

1/1 [==============================] - 0s 407ms/step


(1, 300)

In [110]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_scores = {key : float(cosine_similarity(prediction, encoded_labels[key].reshape(1,-1)))
                     for key in encoded_labels.keys()}

similarity_scores = dict(sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True))

similarity_scores

{'eat': 0.9473861455917358,
 'know nothing': 0.5608543753623962,
 'want': 0.5583537817001343,
 'dont want': 0.5362663269042969,
 'dont feel like': 0.5220218896865845,
 'we will see': 0.5127038955688477,
 'will go': 0.5093497037887573,
 'make believe': 0.5053229331970215,
 'dont like': 0.5034162402153015,
 'i love you': 0.4983612596988678,
 'do do': 0.498063325881958,
 'get': 0.4977402985095978,
 'let go': 0.4945814311504364,
 'dont need': 0.4918353855609894,
 'dont know': 0.490772545337677,
 'shame on you': 0.48980724811553955,
 'let know': 0.48976266384124756,
 'get in bed': 0.48916810750961304,
 'let me see': 0.48881661891937256,
 'easy to do': 0.48881420493125916,
 'spot something': 0.487379789352417,
 'get off': 0.48481300473213196,
 'nothing at all': 0.4830259680747986,
 'try': 0.482712984085083,
 'not a thing': 0.48121514916419983,
 'can': 0.4807920753955841,
 'lose': 0.4793091416358948,
 'know': 0.4789416790008545,
 'absolutely nothing': 0.4783417582511902,
 'come out': 0.478184

In [119]:
predictions = model.predict(X_test)

accurate = 0
for i in tqdm(range(len(predictions)), ncols=100):

    similarity_scores = {
        key: float(cosine_similarity(predictions[i].reshape(1, -1), encoded_labels[key].reshape(1, -1)))
        for key in encoded_labels.keys()
    }

    most_similar = sorted(similarity_scores.items(), key=lambda item: item[1], reverse=True)[0][0]
    
    if most_similar == Y_test[i]:
        accurate += 1

print(f'Accuracy: {(accurate / len(Y_test)):.2f}')

29/29 [==============================] - 1s 41ms/step


100%|█████████████████████████████████████████████████████████████| 907/907 [01:10<00:00, 12.94it/s]

Accuracy: 0.28
